# **Prerequisites**

In [ ]:
!pip install h3

In [ ]:
!pip install geopandas

In [ ]:
!pip install geojson

In [481]:
import pandas as pd
import numpy as np
import h3
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import plotly.graph_objects as go

# **Preprocessing**

In [482]:
dz=pd.read_excel("/content/drive/MyDrive/Datasets/hackathon_bangalore_poi.xlsx")
dz.head()

,source,poi_code,name,poi_type,lat,long,address,city,state,country,pin_code,brand,Unnamed: 12
0,OSM,POI_1003_6612393918,Parle,shop.food.bakery,13.049754,77.492895,NaN,Bangalore,Karnataka,India,NaN,NaN,NaN
1,OSM,POI_1003_6613030600,The Black Pearl,food.restaurant,12.939213,77.694700,NaN,Bangalore,Karnataka,India,NaN,NaN,NaN
2,OSM,POI_1003_4668296373,Temple,NaN,12.940448,77.693816,NaN,Bangalore,Karnataka,India,NaN,NaN,NaN
3,OSM,POI_1003_3771995983,Kadubeesanahalli Underpass,NaN,12.93945,77.695305,NaN,Bangalore,Karnataka,India,NaN,NaN,NaN
4,OSM,POI_1003_2413350412,NaN,health.wellbeing.swimming_pool,12.94066,77.693607,NaN,Bangalore,Karnataka,India,NaN,NaN,NaN


In [682]:
df=dz.copy()

In [683]:
df.columns

Index(['source', 'poi_code', 'name', 'poi_type', 'lat', 'long', 'address',
       'city', 'state', 'country', 'pin_code', 'brand', 'Unnamed: 12'],
      dtype='object')

In [684]:
df.shape

(30417, 13)

In [685]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30417 entries, 0 to 30416
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   source       30417 non-null  object 
 1   poi_code     30417 non-null  object 
 2   name         22151 non-null  object 
 3   poi_type     17314 non-null  object 
 4   lat          30404 non-null  object 
 5   long         30417 non-null  float64
 6   address      1466 non-null   object 
 7   city         30393 non-null  object 
 8   state        30417 non-null  object 
 9   country      30417 non-null  object 
 10  pin_code     5331 non-null   object 
 11  brand        864 non-null    object 
 12  Unnamed: 12  0 non-null      float64
dtypes: float64(2), object(11)
memory usage: 3.0+ MB


In [686]:
df.isna().sum()

source             0
poi_code           0
name            8266
poi_type       13103
lat               13
long               0
address        28951
city              24
state              0
country            0
pin_code       25086
brand          29553
Unnamed: 12    30417
dtype: int64

In [687]:
zz=df['lat'].tolist()
l=[]
for i in zz:
  if isinstance(i,str):
    l.append(i) 
l

['food.restaurant',
 'health.clinic',
 'service.finance.bank',
 'shop.supermarket',
 'service.finance.bank',
 'service.finance.atm',
 'shop.fashion',
 'shop.occasion.gifts',
 'shop.cosmetics.beauty',
 'service.finance.bank',
 'service.finance.bank']

In [688]:
df['lat'] = df['lat'].replace(l, np.nan)

In [689]:
df['lat']=df['lat'].astype(float)

In [690]:
df[df.duplicated(keep=False)]

,source,poi_code,name,poi_type,lat,long,address,city,state,country,pin_code,brand,Unnamed: 12


In [691]:
df["Unnamed: 12"].unique()

array([nan])

In [692]:
df.drop("Unnamed: 12",axis=1,inplace=True)

In [693]:
# df.loc[df['lat']=='food.restaurant']
# df.loc[df['lat']=='health.clinic']
# df.index[df['lat']=='service.finance.bank'].tolist()
# df.drop([5501, 7425, 20147, 23234,1734,2502],inplace=True)
# df = df.reset_index()

In [694]:
df

,source,poi_code,name,poi_type,lat,long,address,city,state,country,pin_code,brand
0,OSM,POI_1003_6612393918,Parle,shop.food.bakery,13.049754,77.492895,NaN,Bangalore,Karnataka,India,NaN,NaN
1,OSM,POI_1003_6613030600,The Black Pearl,food.restaurant,12.939213,77.694700,NaN,Bangalore,Karnataka,India,NaN,NaN
2,OSM,POI_1003_4668296373,Temple,NaN,12.940448,77.693816,NaN,Bangalore,Karnataka,India,NaN,NaN
3,OSM,POI_1003_3771995983,Kadubeesanahalli Underpass,NaN,12.939450,77.695305,NaN,Bangalore,Karnataka,India,NaN,NaN
4,OSM,POI_1003_2413350412,NaN,health.wellbeing.swimming_pool,12.940660,77.693607,NaN,Bangalore,Karnataka,India,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
30412,OSM,POI_1003_3740962178,NaN,NaN,12.976237,77.611640,NaN,Bangalore,Karnataka,India,NaN,NaN
30413,OSM,POI_1003_3740962179,NaN,NaN,12.976275,77.611514,NaN,Bangalore,Karnataka,India,NaN,NaN
30414,OSM,POI_1003_3740962181,NaN,NaN,12.976317,77.611396,NaN,Bangalore,Karnataka,India,NaN,NaN
30415,OSM,POI_1003_3742106925,Soudhamani Estate NSB,NaN,12.821388,77.511637,NaN,Bangalore,Karnataka,India,560082,NaN


In [695]:
df.isna().sum()

source          0
poi_code        0
name         8266
poi_type    13103
lat            24
long            0
address     28951
city           24
state           0
country         0
pin_code    25086
brand       29553
dtype: int64

In [696]:
df.shape[0]

30417

In [697]:
df.dropna(subset=['lat','state'],inplace =True)

In [698]:
# df = df.dropna(thresh=len(df.columns)-1)

In [699]:
df

,source,poi_code,name,poi_type,lat,long,address,city,state,country,pin_code,brand
0,OSM,POI_1003_6612393918,Parle,shop.food.bakery,13.049754,77.492895,NaN,Bangalore,Karnataka,India,NaN,NaN
1,OSM,POI_1003_6613030600,The Black Pearl,food.restaurant,12.939213,77.694700,NaN,Bangalore,Karnataka,India,NaN,NaN
2,OSM,POI_1003_4668296373,Temple,NaN,12.940448,77.693816,NaN,Bangalore,Karnataka,India,NaN,NaN
3,OSM,POI_1003_3771995983,Kadubeesanahalli Underpass,NaN,12.939450,77.695305,NaN,Bangalore,Karnataka,India,NaN,NaN
4,OSM,POI_1003_2413350412,NaN,health.wellbeing.swimming_pool,12.940660,77.693607,NaN,Bangalore,Karnataka,India,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
30412,OSM,POI_1003_3740962178,NaN,NaN,12.976237,77.611640,NaN,Bangalore,Karnataka,India,NaN,NaN
30413,OSM,POI_1003_3740962179,NaN,NaN,12.976275,77.611514,NaN,Bangalore,Karnataka,India,NaN,NaN
30414,OSM,POI_1003_3740962181,NaN,NaN,12.976317,77.611396,NaN,Bangalore,Karnataka,India,NaN,NaN
30415,OSM,POI_1003_3742106925,Soudhamani Estate NSB,NaN,12.821388,77.511637,NaN,Bangalore,Karnataka,India,560082,NaN


In [501]:
df=df.fillna(value={"name":'name',"poi_type":'poi_type'})

In [502]:
df_new = df[['poi_code', 'name','poi_type','lat','long','city','state','country']]

In [503]:
df_new.isnull().sum()

poi_code    0
name        0
poi_type    0
lat         0
long        0
city        0
state       0
country     0
dtype: int64

In [504]:
df_new.shape[0]

30393

In [506]:
df_new.reset_index(drop=True)

,poi_code,name,poi_type,lat,long,city,state,country
0,POI_1003_6612393918,Parle,shop.food.bakery,13.049754,77.492895,Bangalore,Karnataka,India
1,POI_1003_6613030600,The Black Pearl,food.restaurant,12.939213,77.694700,Bangalore,Karnataka,India
2,POI_1003_4668296373,Temple,poi_type,12.940448,77.693816,Bangalore,Karnataka,India
3,POI_1003_3771995983,Kadubeesanahalli Underpass,poi_type,12.939450,77.695305,Bangalore,Karnataka,India
4,POI_1003_2413350412,name,health.wellbeing.swimming_pool,12.940660,77.693607,Bangalore,Karnataka,India
...,...,...,...,...,...,...,...,...
30388,POI_1003_3740962178,name,poi_type,12.976237,77.611640,Bangalore,Karnataka,India
30389,POI_1003_3740962179,name,poi_type,12.976275,77.611514,Bangalore,Karnataka,India
30390,POI_1003_3740962181,name,poi_type,12.976317,77.611396,Bangalore,Karnataka,India
30391,POI_1003_3742106925,Soudhamani Estate NSB,poi_type,12.821388,77.511637,Bangalore,Karnataka,India


# **EDA**

In [701]:
zx=df.dropna(subset=["poi_type"])

In [705]:
z1=zx.groupby('poi_type').count()

In [712]:
z1=z1.reset_index()

In [714]:
s=pd.DataFrame(z1)

In [715]:
s.head()

,index,poi_type,source,poi_code,name,lat,long,address,city,state,country,pin_code,brand
0,0,auto.bicycle,49,49,44,49,49,0,49,49,49,3,1
1,1,auto.bicycle.public_repair,4,4,1,4,4,0,4,4,4,0,0
2,2,auto.bicycle.rental,38,38,15,38,38,0,38,38,38,0,0
3,3,auto.car.dealership,129,129,124,129,129,0,129,129,129,8,35
4,4,auto.car.parts,30,30,30,30,30,0,30,30,30,0,0


In [721]:
fig = px.bar(s, x='source', y='poi_type', title='Population of Brands aroud the market')
fig.show()

In [508]:
gdf = gpd.GeoDataFrame(
    df_new, geometry=gpd.points_from_xy(df_new.long, df_new.lat)
)

In [509]:
gdf

,poi_code,name,poi_type,lat,long,city,state,country,geometry
0,POI_1003_6612393918,Parle,shop.food.bakery,13.049754,77.492895,Bangalore,Karnataka,India,POINT (77.49289 13.04975)
1,POI_1003_6613030600,The Black Pearl,food.restaurant,12.939213,77.694700,Bangalore,Karnataka,India,POINT (77.69470 12.93921)
2,POI_1003_4668296373,Temple,poi_type,12.940448,77.693816,Bangalore,Karnataka,India,POINT (77.69382 12.94045)
3,POI_1003_3771995983,Kadubeesanahalli Underpass,poi_type,12.939450,77.695305,Bangalore,Karnataka,India,POINT (77.69531 12.93945)
4,POI_1003_2413350412,name,health.wellbeing.swimming_pool,12.940660,77.693607,Bangalore,Karnataka,India,POINT (77.69361 12.94066)
...,...,...,...,...,...,...,...,...,...
30412,POI_1003_3740962178,name,poi_type,12.976237,77.611640,Bangalore,Karnataka,India,POINT (77.61164 12.97624)
30413,POI_1003_3740962179,name,poi_type,12.976275,77.611514,Bangalore,Karnataka,India,POINT (77.61151 12.97628)
30414,POI_1003_3740962181,name,poi_type,12.976317,77.611396,Bangalore,Karnataka,India,POINT (77.61140 12.97632)
30415,POI_1003_3742106925,Soudhamani Estate NSB,poi_type,12.821388,77.511637,Bangalore,Karnataka,India,POINT (77.51164 12.82139)


In [510]:
H3_res = 8
def geo_to_h3(row):
  return h3.geo_to_h3(lat=row.lat,lng=row.long,resolution = H3_res)

In [511]:
gdf['h3_cell'] = gdf.apply(geo_to_h3,axis=1)

In [512]:
gdf

,poi_code,name,poi_type,lat,long,city,state,country,geometry,h3_cell
0,POI_1003_6612393918,Parle,shop.food.bakery,13.049754,77.492895,Bangalore,Karnataka,India,POINT (77.49289 13.04975),886014598dfffff
1,POI_1003_6613030600,The Black Pearl,food.restaurant,12.939213,77.694700,Bangalore,Karnataka,India,POINT (77.69470 12.93921),8861892097fffff
2,POI_1003_4668296373,Temple,poi_type,12.940448,77.693816,Bangalore,Karnataka,India,POINT (77.69382 12.94045),8861892097fffff
3,POI_1003_3771995983,Kadubeesanahalli Underpass,poi_type,12.939450,77.695305,Bangalore,Karnataka,India,POINT (77.69531 12.93945),8861892097fffff
4,POI_1003_2413350412,name,health.wellbeing.swimming_pool,12.940660,77.693607,Bangalore,Karnataka,India,POINT (77.69361 12.94066),8861892097fffff
...,...,...,...,...,...,...,...,...,...,...
30412,POI_1003_3740962178,name,poi_type,12.976237,77.611640,Bangalore,Karnataka,India,POINT (77.61164 12.97624),8861892e99fffff
30413,POI_1003_3740962179,name,poi_type,12.976275,77.611514,Bangalore,Karnataka,India,POINT (77.61151 12.97628),8861892e99fffff
30414,POI_1003_3740962181,name,poi_type,12.976317,77.611396,Bangalore,Karnataka,India,POINT (77.61140 12.97632),8861892e9bfffff
30415,POI_1003_3742106925,Soudhamani Estate NSB,poi_type,12.821388,77.511637,Bangalore,Karnataka,India,POINT (77.51164 12.82139),886014504dfffff


In [513]:
hg = (gdf
                          .groupby('h3_cell')
                          .poi_code
                          .agg(list)
                          .to_frame("ids")
                          .reset_index())

hg['count'] =(hg['ids']
                      .apply(lambda poi_code:len(poi_code)))

In [514]:
hg.sort_values('count',ascending=False)

,h3_cell,ids,count
1294,8861892ec3fffff,"[POI_1003_4914346837, POI_1003_3556341348, POI...",923
1290,8861892eb9fffff,"[POI_1003_6897568999, POI_1003_3937163457, POI...",841
1302,8861892ed5fffff,"[POI_1003_6944256063, POI_1003_5265720466, POI...",813
1298,8861892ecbfffff,"[POI_1003_6618583185, POI_1003_6815320086, POI...",523
896,8861892537fffff,"[POI_1003_6631824072, POI_1003_4445603184, POI...",455
...,...,...,...
566,886016946dfffff,[POI_1003_2456829293],1
567,8860169483fffff,[POI_1003_686434003],1
568,8860169489fffff,[POI_1003_3823758621],1
569,88601694c3fffff,[POI_1003_4403238599],1


In [515]:
def add_geometry(row):
  points = h3.h3_to_geo_boundary(row['h3_cell'], True)
  return Polygon(points)

hg['geometry'] = (hg.apply(add_geometry,axis=1))

In [516]:
hg

,h3_cell,ids,count,geometry
0,886014402dfffff,[POI_1003_2595197397],1,POLYGON ((77.38395293796368 12.961672105038087...
1,8860144031fffff,[POI_1003_4401556763],1,POLYGON ((77.35442235125848 12.954276994846946...
2,886014403dfffff,[POI_1003_4401556761],1,"POLYGON ((77.3628242470442 12.954255043253395,..."
3,8860144041fffff,[POI_1003_3223411551],1,POLYGON ((77.38755262621464 12.924293590220556...
4,8860144051fffff,[POI_1003_2593539227],1,POLYGON ((77.36642554799491 12.916880739365125...
...,...,...,...,...
1421,88618936d7fffff,"[POI_1003_4431297026, POI_1003_3250302802]",2,"POLYGON ((77.7123019840338 12.713464482206238,..."
1422,88618936ddfffff,[POI_1003_4436681197],1,POLYGON ((77.72481708583017 12.705907215851104...
1423,88618936e1fffff,[POI_1003_2026392483],1,POLYGON ((77.74219649175431 12.735777686680372...
1424,8861893713fffff,"[POI_1003_6944525387, POI_1003_313271644, POI_...",4,POLYGON ((77.78080249591646 12.773032904224381...


In [517]:
from geojson import Feature, Point, FeatureCollection, Polygon

def hexagons_dataframe_to_geojson(df_hex, hex_id_field,geometry_field, value_field,file_output = None):

    list_features = []

    for i, row in df_hex.iterrows():
        feature = Feature(geometry = row[geometry_field],
                          id = row[hex_id_field],
                          properties = {"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    else :
      return feat_collection

In [520]:
geojson_obj = (hexagons_dataframe_to_geojson(hg,hex_id_field='h3_cell',value_field='count',geometry_field='geometry'))

In [522]:
import plotly.express as px

In [524]:
fig = (px.choropleth_mapbox(
                    hg, 
                    geojson=geojson_obj, 
                    locations='h3_cell', 
                    color='count',
                    color_continuous_scale="Viridis",
                    range_color=(0,hg['count'].mean()),mapbox_style='carto-positron',
                    zoom=7,
                    center = {"lat": 12.971767302592704, "lon": 77.6112392662554},
                    opacity=0.7,
                    labels={'count':'Count of spaces occupied in each Locality'}))
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# **`More insights using the prices dataset `**

In [525]:
data=pd.read_csv("/content/drive/MyDrive/Datasets/Blr_residential_sample.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning:

Columns (38,166,168,200,209,228,230,232,233,235,236,249,251,252) have mixed types.Specify dtype option on import or set low_memory=False.



In [527]:
data.head()

,SPID,PROFILEID,PROP_ID,PHOTO_URL,MEDIUM_PHOTO_URL,PREFERENCE,DESCRIPTION,PROPERTY_TYPE,CITY,LOCALITY,...,IS_CREDIT_LISTING,SHOW_ONLY_START_PRICE,OTHER_CITY,FOOD_AVAILABILITY,AVAILABILITY_DATE__U,MODIFY_DATE__U,FD,LOGO_URL,Latitude,Longitude
0,64808356,39096545,O64808356,https://mediacdn.99acres.com/media1/19104/2/38...,https://mediacdn.99acres.com/media1/19104/2/38...,S,This well-known project offers 3 BHK apartment...,Residential Apartment,Bangalore Central,Cooke Town,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.999820,77.625833
1,63759512,13651131,F63759512,https://mediacdn.99acres.com/media1/18903/1/37...,https://mediacdn.99acres.com/media1/18903/1/37...,S,Experience a new style of living with Brigade ...,Residential Apartment,Bangalore North,Kogilu,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.104031,77.608912
2,60610644,6113011,O60610644,https://mediacdn.99acres.com/media1/18766/19/3...,https://mediacdn.99acres.com/media1/18766/19/3...,S,This is your chance to own a 2 bhk residential...,Residential Apartment,Bangalore South,Maruthi Nagar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.926749,77.613164
3,64126420,6113011,Q64126420,https://mediacdn.99acres.com/media1/18921/11/3...,https://mediacdn.99acres.com/media1/18921/11/3...,S,This is your chance to own a 2 bhk residential...,Residential Apartment,Bangalore East,CV Raman Nagar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.983570,77.662180
4,64125950,6113011,P64125950,https://mediacdn.99acres.com/media1/18858/14/3...,https://mediacdn.99acres.com/media1/18858/14/3...,S,This is your chance to own a 3 bhk residential...,Residential Apartment,Bangalore East,CV Raman Nagar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.983570,77.662180


In [528]:
data.shape

(9649, 255)

In [529]:
data.columns

Index(['SPID', 'PROFILEID', 'PROP_ID', 'PHOTO_URL', 'MEDIUM_PHOTO_URL',
       'PREFERENCE', 'DESCRIPTION', 'PROPERTY_TYPE', 'CITY', 'LOCALITY',
       ...
       'IS_CREDIT_LISTING', 'SHOW_ONLY_START_PRICE', 'OTHER_CITY',
       'FOOD_AVAILABILITY', 'AVAILABILITY_DATE__U', 'MODIFY_DATE__U', 'FD',
       'LOGO_URL', 'Latitude', 'Longitude'],
      dtype='object', length=255)

In [535]:
new_df=data[['SPID','PROPERTY_TYPE','LOCALITY','PRICE_PER_UNIT_AREA','PRICE_SQFT','Latitude','Longitude']]

In [536]:
new_df.isnull().sum()

SPID                   0
PROPERTY_TYPE          0
LOCALITY               0
PRICE_PER_UNIT_AREA    6
PRICE_SQFT             6
Latitude               0
Longitude              0
dtype: int64

In [537]:
new_df.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [538]:
new_df

,SPID,PROPERTY_TYPE,LOCALITY,PRICE_PER_UNIT_AREA,PRICE_SQFT,Latitude,Longitude
0,64808356,Residential Apartment,Cooke Town,23059.0,23059.0,12.999820,77.625833
1,63759512,Residential Apartment,Kogilu,11135.0,11135.0,13.104031,77.608912
2,60610644,Residential Apartment,Maruthi Nagar,4171.0,4171.0,12.926749,77.613164
3,64126420,Residential Apartment,CV Raman Nagar,5391.0,5391.0,12.983570,77.662180
4,64125950,Residential Apartment,CV Raman Nagar,10507.0,10507.0,12.983570,77.662180
...,...,...,...,...,...,...,...
9644,64587642,Residential Land,Sector 3 HSR Layout,22000.0,22000.0,12.909432,77.640313
9645,64586936,Residential Land,Sector 3 HSR Layout,37500.0,37500.0,12.909432,77.640313
9646,64586772,Residential Land,Sector 3 HSR Layout,17500.0,17500.0,12.909432,77.640313
9647,64586542,Residential Land,Sector 6 HSR Layout,22000.0,22000.0,12.913887,77.628533


In [539]:
gdf = gpd.GeoDataFrame(
    new_df, geometry=gpd.points_from_xy(new_df.Latitude, new_df.Longitude)
)

In [540]:
H3_res = 9
def geo_to_h3(row):
  return h3.geo_to_h3(lat=row.Latitude,lng=row.Longitude,resolution = H3_res)

In [541]:
gdf['h3_cell'] = gdf.apply(geo_to_h3,axis=1)

In [ ]:
d=dict(zip(new_df.SPID, new_df.PRICE_PER_UNIT_AREA))
d

In [549]:
def avg_price(l):
  sum=0
  count=0
  for i in l:
    sum+=d[i]
  return(sum/len(l))
avg_price([64032076,64023120,64023050])

4474.666666666667

In [550]:
hg = (gdf.groupby('h3_cell')
                          .SPID
                          .agg(list)
                          .to_frame("ids")
                          .reset_index())

hg['count'] =(hg['ids']
                      .apply(lambda SPID:len(SPID)))

hg['avg_price'] =(hg['ids']
                      .apply(avg_price))

In [551]:
hg.sort_values('count',ascending=False)

,h3_cell,ids,count,avg_price
605,8960145b483ffff,"[63932882, 58998970, 65007122, 64991554, 54765...",215,71166.800000
2077,8961892c81bffff,"[62644872, 59315848, 62873738, 64958496, 64956...",137,35668.160584
18,89015189e2fffff,"[63806654, 62625628, 63726068, 63308872, 64989...",114,16782.745614
15,89015189a5bffff,"[65012780, 64944170, 64930334, 61021646, 64091...",93,478361.967742
912,89601697183ffff,"[63223062, 63423912, 61358688, 64974202, 64933...",91,8926.175824
...,...,...,...,...
1082,89618921957ffff,[64991776],1,5750.000000
1078,89618921927ffff,[44476651],1,4412.000000
1077,896189218dbffff,[55186630],1,4642.000000
1076,896189218d3ffff,[63725902],1,5600.000000


In [552]:
from shapely.geometry import Polygon

def add_geometry(row):
  points = h3.h3_to_geo_boundary(row['h3_cell'], True)
  return Polygon(points)

hg['geometry'] = (hg.apply(add_geometry,axis=1))

In [553]:
from geojson import Feature, Point, FeatureCollection, Polygon

def hexagons_dataframe_to_geojson(df_hex, hex_id_field,geometry_field, value_field,file_output = None):

    list_features = []

    for i, row in df_hex.iterrows():
        feature = Feature(geometry = row[geometry_field],
                          id = row[hex_id_field],
                          properties = {"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    else :
      return feat_collection

In [554]:
geojson_obj = (hg(hg,hex_id_field='h3_cell',value_field='avg_price',geometry_field='geometry'))

In [555]:
fig = (px.choropleth_mapbox(
                    hg, 
                    geojson=geojson_obj, 
                    locations='h3_cell', 
                    color='avg_price',
                    color_continuous_scale="Viridis",
                    range_color=(0,hg['avg_price'].mean()),mapbox_style='carto-positron',
                    zoom=7,
                    center = {"lat": 12.971767302592704, "lon": 77.6112392662554},
                    opacity=0.7,
                    labels={'Price':'Average Prices of spaces vacated'}))
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [453]:
len(fire_ignitions_g['avg_price'].unique())

2278

In [454]:
# df.rename(columns = {'Latitude':'lat','Longitude':'long'}, inplace = True)
# new=pd.merge(dz,df,on=['lat','long'])
# new1=new[['source', 'poi_code', 'name', 'poi_type', 'lat', 'long', 'address',
#        'city', 'state', 'country', 'pin_code', 'brand','SPID','PROPERTY_TYPE','LOCALITY','PRICE_PER_UNIT_AREA','PRICE_SQFT']]

# **EDA(To understand more about the data)**

In [556]:
l=data.columns.tolist()
print(l)

['SPID', 'PROFILEID', 'PROP_ID', 'PHOTO_URL', 'MEDIUM_PHOTO_URL', 'PREFERENCE', 'DESCRIPTION', 'PROPERTY_TYPE', 'CITY', 'LOCALITY', 'AREA_UNIT', 'TRANSACT_TYPE', 'OWNTYPE', 'BEDROOM_NUM', 'BATHROOM_NUM', 'PRICE_PER_UNIT_AREA', 'AVAILABILITY', 'FURNISH', 'FACING', 'AGE', 'FLOOR_NUM', 'TOTAL_FLOOR', 'FEATURES', 'REGISTER_DATE', 'POSTING_DATE', 'UPDATE_DATE', 'CLASS', 'RES_COM', 'PROP_NAME', 'PROPERTY_NUMBER', 'MIN_AREA', 'MAX_AREA', 'MIN_PRICE', 'MAX_PRICE', 'PRICE_SQFT', 'LISTING', 'PROJ_ID', 'BUILDING_ID', 'COM_AREA_TYPE', 'VERIFIED', 'MASK_CONTACT', 'CONF_ID', 'ASSIGNED_TO', 'RESERVED_PARKING', 'MAP_DETAILS', 'FSL_Data', 'MIN_AREA_SQFT', 'MAX_AREA_SQFT', 'SHOW_BSP', 'IS_NEW_LAUNCH', 'IS_POSTER_RERA_REGISTERED', 'IS_DEALER_RERA_REGISTERED', 'RERA_REGISTRATION_ID', 'FORMATTED', 'HAVEPHOTO', 'AMENITIES', 'ALT_TAG', 'PRODUCT_TYPE', 'FOMO', 'PD_URL', 'SELF_VERIFIED', 'EXPIRY_DATE', 'GROUP_NAME', 'GROUP_META', 'PROPERTY_TYPE__U', 'AREA_UNIT__U', 'POSTING_DATE__U', 'UPDATE_DATE__U', 'EXPIRY_

In [557]:
d=data.copy()

In [558]:
d=d[['Latitude', 'Longitude','LOCALITY', 'AREA_UNIT','PRICE_PER_UNIT_AREA']].dropna()

In [559]:
d.shape

(2248, 5)

In [560]:
fig = px.density_mapbox(data, lat='Latitude', lon='Longitude', z='AGE', radius=10,
                        zoom=3,hover_name='LOCALITY',
                        mapbox_style="stamen-terrain")
fig.show()

In [574]:
# d1=data.drop_duplicates().groupby('LOCALITY').count()
# d1.reset_index()
# fig = px.treemap(d1, path=['SPID','MIN_AREA_SQFT'], values='CLASS')
# fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.show()

In [620]:
dg=data[['SPID','LOCALITY','PRICE_PER_UNIT_AREA','AVAILABILITY','AGE','CLASS','MAX_PRICE','MIN_PRICE','PRICE_SQFT','MIN_AREA_SQFT','MAX_AREA_SQFT','Latitude', 'Longitude']]

In [621]:
dg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9649 entries, 0 to 9648
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   SPID                 9649 non-null   int64  
 1   LOCALITY             9649 non-null   object 
 2   PRICE_PER_UNIT_AREA  9643 non-null   float64
 3   AVAILABILITY         9518 non-null   object 
 4   AGE                  9649 non-null   int64  
 5   CLASS                9649 non-null   object 
 6   MAX_PRICE            9649 non-null   int64  
 7   MIN_PRICE            9649 non-null   int64  
 8   PRICE_SQFT           9643 non-null   float64
 9   MIN_AREA_SQFT        9649 non-null   float64
 10  MAX_AREA_SQFT        9649 non-null   float64
 11  Latitude             9649 non-null   float64
 12  Longitude            9649 non-null   float64
dtypes: float64(6), int64(4), object(3)
memory usage: 980.1+ KB


In [622]:
dg.isnull().sum()

SPID                     0
LOCALITY                 0
PRICE_PER_UNIT_AREA      6
AVAILABILITY           131
AGE                      0
CLASS                    0
MAX_PRICE                0
MIN_PRICE                0
PRICE_SQFT               6
MIN_AREA_SQFT            0
MAX_AREA_SQFT            0
Latitude                 0
Longitude                0
dtype: int64

In [623]:
dg.dropna(inplace=True)

In [624]:
print(l)

['SPID', 'PROFILEID', 'PROP_ID', 'PHOTO_URL', 'MEDIUM_PHOTO_URL', 'PREFERENCE', 'DESCRIPTION', 'PROPERTY_TYPE', 'CITY', 'LOCALITY', 'AREA_UNIT', 'TRANSACT_TYPE', 'OWNTYPE', 'BEDROOM_NUM', 'BATHROOM_NUM', 'PRICE_PER_UNIT_AREA', 'AVAILABILITY', 'FURNISH', 'FACING', 'AGE', 'FLOOR_NUM', 'TOTAL_FLOOR', 'FEATURES', 'REGISTER_DATE', 'POSTING_DATE', 'UPDATE_DATE', 'CLASS', 'RES_COM', 'PROP_NAME', 'PROPERTY_NUMBER', 'MIN_AREA', 'MAX_AREA', 'MIN_PRICE', 'MAX_PRICE', 'PRICE_SQFT', 'LISTING', 'PROJ_ID', 'BUILDING_ID', 'COM_AREA_TYPE', 'VERIFIED', 'MASK_CONTACT', 'CONF_ID', 'ASSIGNED_TO', 'RESERVED_PARKING', 'MAP_DETAILS', 'FSL_Data', 'MIN_AREA_SQFT', 'MAX_AREA_SQFT', 'SHOW_BSP', 'IS_NEW_LAUNCH', 'IS_POSTER_RERA_REGISTERED', 'IS_DEALER_RERA_REGISTERED', 'RERA_REGISTRATION_ID', 'FORMATTED', 'HAVEPHOTO', 'AMENITIES', 'ALT_TAG', 'PRODUCT_TYPE', 'FOMO', 'PD_URL', 'SELF_VERIFIED', 'EXPIRY_DATE', 'GROUP_NAME', 'GROUP_META', 'PROPERTY_TYPE__U', 'AREA_UNIT__U', 'POSTING_DATE__U', 'UPDATE_DATE__U', 'EXPIRY_

In [625]:
dg.shape

(9512, 13)

In [631]:
dg

,SPID,LOCALITY,PRICE_PER_UNIT_AREA,AVAILABILITY,AGE,CLASS,MAX_PRICE,MIN_PRICE,PRICE_SQFT,MIN_AREA_SQFT,MAX_AREA_SQFT,Latitude,Longitude
0,64808356,Cooke Town,23059.0,I,5,A,27420933,21053105,23059.0,87.050111,109.811346,12.999820,77.625833
1,63759512,Kogilu,11135.0,I,6,B,11001380,10199660,11135.0,85.099148,91.788164,13.104031,77.608912
2,60610644,Maruthi Nagar,4171.0,I,2,O,4200000,4200000,4171.0,89.093977,89.093977,12.926749,77.613164
3,64126420,CV Raman Nagar,5391.0,I,3,O,6200000,6200000,5391.0,106.838450,106.838450,12.983570,77.662180
4,64125950,CV Raman Nagar,10507.0,I,2,O,23000000,23000000,10507.0,203.364667,203.364667,12.983570,77.662180
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,64587642,Sector 3 HSR Layout,22000.0,L,0,A,52800000,52800000,22000.0,222.967200,222.967200,12.909432,77.640313
9645,64586936,Sector 3 HSR Layout,37500.0,L,0,A,45000000,45000000,37500.0,111.483600,111.483600,12.909432,77.640313
9646,64586772,Sector 3 HSR Layout,17500.0,L,0,A,42000000,42000000,17500.0,222.967200,222.967200,12.909432,77.640313
9647,64586542,Sector 6 HSR Layout,22000.0,L,0,A,88000000,88000000,22000.0,371.612000,371.612000,12.913887,77.628533


In [638]:
dg['MAX_AREA_SQFT'].mean()

599.6459198264298

In [644]:
v=dg.loc[dg['MAX_AREA_SQFT'] >=dg['MAX_AREA_SQFT'].mean()]

In [646]:
s=v.loc[v['PRICE_PER_UNIT_AREA'] >=v['PRICE_PER_UNIT_AREA'].mean()] 

In [647]:
s

,SPID,LOCALITY,PRICE_PER_UNIT_AREA,AVAILABILITY,AGE,CLASS,MAX_PRICE,MIN_PRICE,PRICE_SQFT,MIN_AREA_SQFT,MAX_AREA_SQFT,Latitude,Longitude
183,63475794,Chikkaballapur,34848000.0,I,0,O,2147483647,2147483647,34848000.0,607028.202000,607028.202000,13.426080,77.730400
395,64981974,Devanahalli,25000000.0,L,0,O,25000000,25000000,25000000.0,4046.854680,4046.854680,13.243287,77.710834
448,58350458,Chintamani,5714285.0,I,0,O,18000000,18000000,5714285.0,12747.592242,12747.592242,13.381818,78.043522
1104,63986474,"Dasanpura, Hobli",41666666.0,I,3,O,250000000,250000000,41666666.0,24281.128080,24281.128080,77.607315,13.048041
2935,63404218,Kanakapura Road,4090909.0,I,6,O,9000000,9000000,4090909.0,8903.080296,8903.080296,12.935800,77.575150
3064,21053618,Murugamalla,2777778.0,I,0,O,20000000,20000000,2777778.0,29137.353696,29137.353696,13.048041,77.607315
3517,63281164,Arehalli,20000000.0,I,0,O,40000000,40000000,20000000.0,8093.709360,8093.709360,12.929320,77.838160
3673,51908302,Tumkur Road,16666666.0,I,0,O,25000000,25000000,16666666.0,6070.282020,6070.282020,13.140142,77.412425
3785,60747654,Kanakapura,3900000.0,L,0,O,7800000,7800000,3900000.0,8093.709360,8093.709360,12.548531,77.419167
4938,57061594,Tippagondanahalli,8900000.0,I,0,O,40000000,40000000,8900000.0,16187.418720,16187.418720,12.971600,77.594560


In [666]:
geometry = [Point(xy) for xy in zip(s['Longitude'], s['Latitude'])]
gdf = GeoDataFrame(s, geometry=geometry)   

In [628]:
c=dg[['AGE','MAX_PRICE','PRICE_SQFT','MIN_AREA_SQFT']]

In [629]:
fig = px.parallel_coordinates(c)
fig.show()

In [675]:
gdf.head()

,SPID,LOCALITY,PRICE_PER_UNIT_AREA,AVAILABILITY,AGE,CLASS,MAX_PRICE,MIN_PRICE,PRICE_SQFT,MIN_AREA_SQFT,MAX_AREA_SQFT,Latitude,Longitude,geometry
183,63475794,Chikkaballapur,34848000.0,I,0,O,2147483647,2147483647,34848000.0,607028.202000,607028.202000,13.426080,77.730400,POINT (77.73040 13.42608)
395,64981974,Devanahalli,25000000.0,L,0,O,25000000,25000000,25000000.0,4046.854680,4046.854680,13.243287,77.710834,POINT (77.71083 13.24329)
448,58350458,Chintamani,5714285.0,I,0,O,18000000,18000000,5714285.0,12747.592242,12747.592242,13.381818,78.043522,POINT (78.04352 13.38182)
1104,63986474,"Dasanpura, Hobli",41666666.0,I,3,O,250000000,250000000,41666666.0,24281.128080,24281.128080,77.607315,13.048041,POINT (13.04804 77.60731)
2935,63404218,Kanakapura Road,4090909.0,I,6,O,9000000,9000000,4090909.0,8903.080296,8903.080296,12.935800,77.575150,POINT (77.57515 12.93580)


In [679]:
fig = go.Figure(data=go.Scattergeo(
        lon = gdf['Longitude'],
        lat = gdf['Latitude'],
        text = gdf['LOCALITY'],
        mode = 'markers',
        marker_color = gdf['PRICE_PER_UNIT_AREA'],
        ))

fig.update_layout(
        title = 'Most Expensive Areas in Bangalore (Hover for Locality names)',
        geo_scope='asia',
    )
fig.show()